# Laboratorio 6: Solución


## Problema de la dieta

La siguiente tabla indica el contenido de varios nutrientes (hemos considerado a la energía también como un nutriente) para un conjunto dado de alimentos, además de la cantidad máxima de porciones diarias que pueden ser consumidas de cada alimento, y el precio de cada porción.

| Alimento   | Porciones diarias | Energía (kcal) | Proteínas (g)|Calcio (mg)|Precio (ctvos)|
| -----------|:-----------------:| --------------:|-------------:|----------:|-------------:|
|    Avena   |       $$4$$       |       $$110$$  |   $$4$$      |    $$2$$  |   $$3$$      |
|    Pollo   |       $$3$$       |       $$205$$  |   $$32$$     |    $$12$$ |   $$24$$     |
|    Huevos  |      $$2$$        |       $$160$$  |   $$13$$     |    $$54$$ |   $$13$$     |
|    Leche   |      $$8$$        |       $$160$$  |   $$8$$      |   $$284$$ |   $$9$$      |
|    Pastel  |      $$2$$        |       $$420$$  |    $$4$$     |   $$22$$  |   $$20$$     |
|    Cerdo   |      $$2$$        |       $$260$$  |    $$14$$    |   $$80$$  |   $$19$$     |

Se conoce que los requerimientos diarios mínimos de nutrientes son de $2000$kcal en energía, $55$g de proteína y $800$mg de calcio. Se desea diseñar una dieta para cubrir estos requerimientos al menor costo, tomando en cuenta que solamente es posible consumir un número entero de porciones de cada alimento. 

Formular este problema como un **programa lineal entero**, escribirlo usando la interfaz Python de Gurobi, resolverlo y mostrar la solución óptima. 


Suponer que los datos del problema están dados en la variable `datos_alimentos`:

In [1]:
# La tupla de datos de cada alimento indica:
# (porciones maximas, energia, proteinas, calcio, precio)
datos_alimentos = {'Avena' : (4, 110, 4, 2, 3),'Pollo' : (3, 205, 32, 12, 24), 'Huevos' : (2, 160, 13, 54, 13), 
                  'Leche' : (8, 160, 8, 284, 9), 'Pastel' : (2, 420, 4, 22, 20), 'Cerdo' : (2, 260, 14, 80, 19)}

Para implementar este modelo, empezamos por importar el módulo de Gurobi y definir el conjunto de alimentos $I$, así como los demás parámetros del modelo, llamando a la función `multidic` :

In [2]:
from gurobipy import *

I, porciones, energia, proteinas, calcio, precio = multidict(datos_alimentos)

Luego creamos el objeto modelo:

In [3]:
m = Model('Problema_de_la_dieta')

Academic license - for non-commercial use only


Ahora creamos variables enteras $x_i$, indexadas por $i \in I$, que representan la cantidad diaria de porciones a consumir de cada alimento. 

Es importante tener en cuenta que estas variables no pueden tomar valores que sobrepasen la cantidad máxima de porciones diarias. Para definir variables acotadas superiormente, se puede emplear el argumento `ub` (*upper bound*) del método `addVars()`.

In [4]:
x = m.addVars(I, vtype = GRB.INTEGER, name="x",ub=porciones)

Añadimos ahora la función objetivo: Minimizar los costos asociados a la dieta.

In [5]:
# construir sumatorio precio_i * x_i (i en I) usando el metodo prod
m.setObjective(x.prod(precio, '*'), GRB.MINIMIZE)

Finalmente, implementamos las restricciones del modelo:

Dado que la cantidad total de un nutriente es la suma de las cantidades de los nutrientes en todos los alimentos, se deben cumplir las siguientes restricciones:
1. Requerimiento mínimo de energía.

In [6]:
# construir sumatorio energia_i * x_i (i en I) usando el metodo prod
m.addConstr(x.prod(energia, '*') >= 2000, "req_ener")

<gurobi.Constr *Awaiting Model Update*>

2. Requerimiento mínimo de proteínas.

In [7]:
# construir sumatorio proteina_i * x_i (i en I) usando un generador
m.addConstr(x.prod(proteinas, '*') >= 55, "req_ener")

<gurobi.Constr *Awaiting Model Update*>

3. Requerimiento mínimo de calcio.

In [8]:
# construir sumatorio calcio_i * x_i (i en I) usando un generador
m.addConstr(x.prod(calcio, '*') >= 800, "req_prot")

<gurobi.Constr *Awaiting Model Update*>

Llamamos ahora a `optimize` para resolver el modelo.

In [9]:
m.optimize()

Optimize a model with 3 rows, 6 columns and 18 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 4e+02]
  Objective range  [3e+00, 2e+01]
  Bounds range     [2e+00, 8e+00]
  RHS range        [6e+01, 2e+03]
Found heuristic solution: objective 136.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.14s
Presolved: 2 rows, 6 columns, 12 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: objective 9.250000e+01, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   92.50000    0    1  136.00000   92.50000  32.0%     -    0s
H    0     0                      97.0000000   92.50000  4.64%     -    0s

Cutting planes:
  Gomory: 1

Explored 1 nodes (1 simplex iterations) in 0.37 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 97 136 

O

Mostrar las porciones por alimento requeridas de acuerdo a la dieta óptima:

In [10]:
print('Alimento\t Porción')
print('------------------------')
for i in I:
    if x[i].x >= 0.99:
        print('{}\t\t {}'.format(i,x[i].x))

Alimento	 Porción
------------------------
Pastel		 2.0
Avena		 4.0
Leche		 5.0
